In [ ]:
# default_exp models.DeepConvRNN_dev

# DeepConvRNN

> This is an unofficial PyTorch implementation by Ignacio Oguiza - oguiza@gmail.com based on:
- Ordóñez, F. J., & Roggen, D. (2016). Deep convolutional and lstm recurrent neural networks for multimodal wearable activity recognition. Sensors, 16(1), 115.
- Official DeepConvLSTM theano implementation: https://github.com/sussexwearlab/DeepConvLSTM

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
import torch
import torch.nn as nn
import torch.nn.functional as F
import math


def same_padding2d(input_dim, kernel_size, stride=1, dilation=1):
    pad = (stride * (input_dim - 1) - input_dim + kernel_size +
           (kernel_size - 1) * (dilation - 1)) / 2
    return (0, 0, math.ceil(pad), int(pad))


class ShuffleRNN(nn.Module):
    def __init__(self,
                 input_size,
                 hidden_size=128,
                 num_layers=1,
                 dropout=0.0,
                 bidirectional=False,
                 cell_type='LSTM'):
        super().__init__()
        self.cell_type = cell_type
        if self.cell_type == "GRU":
            cell = nn.GRU
        elif self.cell_type == "LSTM":
            cell = nn.LSTM
        self.rnn = cell(input_size=input_size,
                        hidden_size=hidden_size,
                        num_layers=num_layers,
                        dropout=dropout,
                        bidirectional=bidirectional)

    def forward(self, x):
        if self.cell_type == "LSTM":
            out, (_, _) = self.rnn(x)
        elif self.cell_type == "GRU":
            out, _ = self.rnn(x)
        last_out = out[-1]
        return last_out


class ConvLayer2d(nn.Module):
    def __init__(self,
                 in_channels=1,
                 n_channels=128,
                 kernel_size=8,
                 same_padding=False,
                 dropout=0.0):
        super().__init__()
        self.conv = nn.Conv2d(in_channels=in_channels,
                              out_channels=n_channels,
                              kernel_size=kernel_size)
        self.bn = nn.BatchNorm2d(n_channels)
        self.dropout = nn.Dropout(dropout)
        self.kernel_size = kernel_size
        self.same_padding = same_padding

    def forward(self, x):
        if self.same_padding:
            m = nn.ZeroPad2d(same_padding2d(x.shape[2], self.kernel_size[0]))
            x = m(x)
        x = self.conv(x)
        x = self.bn(x)
        x = F.relu(x)
        x = self.dropout(x)
        return x


class DeepConvRNN(nn.Module):
    def __init__(self,
                 features,
                 n_classes,
                 conv_layers=4 * [64],
                 kernel_sizes=4 * [5],
                 same_padding=True,
                 rnn_layers=2 * [128],
                 rnn_bidirectional=False,
                 rnn_dropout=0.0,
                 fc_dropout=0.0,
                 cell_type='LSTM'):
        super(DeepConvRNN, self).__init__()

        self.features = features
        self.n_classes = n_classes
        self.conv_layers = conv_layers
        self.n_conv_layers = len(conv_layers)
        self.kernel_sizes = kernel_sizes
        self.ks = list(zip(kernel_sizes, len(kernel_sizes) * [1]))
        conv_cells = []
        for i in range(self.n_conv_layers):
            if i == 0:
                input_dim = 1
            else:
                input_dim = self.conv_layers[i - 1]

            cell = ConvLayer2d(input_dim,
                               self.conv_layers[i],
                               self.ks[i],
                               same_padding=same_padding)
            name = 'ConvLayer_' + str(i).zfill(2)
            setattr(self, name, cell)
            conv_cells.append(getattr(self, name))

        self.conv_cells = conv_cells
        self.rnn_hidden_size = rnn_layers[0]
        self.rnn_num_layers = len(rnn_layers)
        self.rnn_dropout = rnn_dropout
        self.rnn_bidirectional = rnn_bidirectional
        self.cell_type = cell_type
        self.fc_dropout = nn.Dropout(fc_dropout)
        self.rnn = ShuffleRNN(self.conv_layers[-1] * self.features,
                              hidden_size=self.rnn_hidden_size,
                              num_layers=self.rnn_num_layers,
                              bidirectional=self.rnn_bidirectional,
                              dropout=rnn_dropout,
                              cell_type=self.cell_type)

        if rnn_bidirectional:
            self.fc = nn.Linear(self.rnn_hidden_size * 2, n_classes)
        else:
            self.fc = nn.Linear(self.rnn_hidden_size, n_classes)
        self.arch = 'DeepConv' + cell_type

    def forward(self, x):
        if x.ndim < 4: x = x.unsqueeze(1)
        print(x.shape)
        x = x.permute(0, 1, 3, 2)
        for layer_idx in range(self.n_conv_layers):
            cell = self.conv_cells[layer_idx]
            x = cell(x)
        x = x.permute(2, 0, 1, 3).contiguous()
        x = x.view(x.shape[0], x.shape[1], -1) 
        x = self.rnn(x)
        x = self.fc_dropout(x)
        x = self.fc(x)
        return x

    def predict(self, inputs):
        logits = self.forward(inputs)
        softmax_layer = torch.nn.Softmax(dim=1)
        probs = softmax_layer(logits)
        _, idx = torch.max(probs, 1)
        return idx


from functools import partial
DeepConvLSTM = partial(DeepConvRNN,
                       conv_layers=[64, 64, 64, 64],
                       kernel_sizes=[5, 5, 5, 5],
                       same_padding=True,
                       rnn_layers=[128, 128],
                       rnn_bidirectional=False,
                       rnn_dropout=0.0,
                       fc_dropout=0.0,
                       cell_type='LSTM')
setattr(DeepConvLSTM, '__name__', 'DeepConvLSTM')

DeepConvGRU = partial(DeepConvRNN,
                      conv_layers=[64, 64, 64, 64],
                      kernel_sizes=[5, 5, 5, 5],
                      same_padding=True,
                      rnn_layers=[128, 128],
                      rnn_bidirectional=False,
                      rnn_dropout=0.0,
                      fc_dropout=0.0,
                      cell_type='GRU')
setattr(DeepConvGRU, '__name__', 'DeepConvGRU')

In [ ]:
xb = torch.rand(16, 1, 3, 128)
DeepConvLSTM(3, 2)(xb).shape

torch.Size([16, 1, 3, 128])


torch.Size([16, 2])

In [ ]:
#hide
from save_nb import *
from fastai2.data.all import *
from nbdev.export import notebook2script
save_nb()
notebook2script()
test_eq(last_saved() < 10, True)

<IPython.core.display.Javascript object>


Current notebook saved.

Converted 000_utils.ipynb.
Converted 001_core.ipynb.
Converted 002_data.ipynb.
Converted 100_layers.ipynb.
Converted 101_ResNet.ipynb.
Converted 102_InceptionTime.ipynb.
Converted 103_DeepConvRNN_dev.ipynb.
Converted index.ipynb.

core.py                        saved          0 s ago
utils.py                       saved          1 s ago
data.py                        saved          0 s ago
DeepConvRNN_dev.py             saved          0 s ago
ResNet.py                      saved          0 s ago
InceptionTime.py               saved          0 s ago
layers.py                      saved          0 s ago

Total elapsed time 2 s
05-04-2020 07:20:00
